<a href="https://colab.research.google.com/github/lustraka/Data_Analysis_Workouts/blob/main/Analyse_Twitter_Data/wrangle_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## WeRateDogs Data Wrangle Report
This report briefly describes wrangling efforts focused on data from an archive of WeRateDogs Twitter account. **WeRateDogs** Twitter account rates people's dogs with a humorous comment about the dog. The account was started in 2015 by college student Matt Nelson, and has received international media attention both for its popularity and for the attention drawn to social media copyright law (see [Wikipedia](https://en.wikipedia.org/wiki/WeRateDogs)).

## Gather Data
Data is gathered from three sources:

![Gather Data Diagram](http://www.plantuml.com/plantuml/png/ZP7DIiD04CVl-nH33ZOUsfDua1egYE3D1n32a9sq8yacxapwWVZkxYOf2PP2Bu7_x7pyVcGZYbwxkZ9OLSsXvYExhJ4bLmI97bYLV8w-sF6USd8xT0NPLI7x153WMeCXfwmdi2N2UlsAYcyUQyeo-35w8DASBJm1EtryCB_csXfOBZD8FA7DmyNqFeaBZIcQkcqSEGLFtns9oiZBDMufRprPBfGR9oiTGIzU9aA-XV1y298jQThvAh14kCkz5vL_4aBjy-T25xqCpsNYsVOKQ5u-thDiL1fOlgHnAprgxpALPiCnJavwYzDuBnhbegYTfEUkqIwnNTrA6bA_FdvW7Uf3srHBgWQjfEUfqZTr7lM7FwOcMB8leWwAa0ZSGKZeUfsXP8mjCOwFEKIUHGXPvyyRSW6k-WC0)

### Enhanced Twitter Archive
The WeRateDogs Twitter archive contains basic tweet data for all tweets from the November 15th, 2015 to the August 17th, 2017. The each tweet's text had been used to extract rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) to enhance this archive.

The enhanced archive is stored in the `twitter_archive_enhanced.csv` file and read directly by `pd.read_csv()` method to the `dfa` DataFrame.

### Image Prediction File
Every image in the WeRateDogs Twitter archive had been run through a machine learning algorithm to classify breeds of dogs. The `image_prediction.tsv` file contains top three predictions alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction.

The image prediction file is first downloaded using the `requests` library and then loaded with `pd.read_csv()` method to the `dfi` DataFrame.

### Twitter API
IDs of tweets from Twitter archive was used to access missing valuable data, namely the number of likes (*favorite_count*) and the number of retweets (*retweet_count*), via Twitter's API. Before we could ran our API querying code, we needed to set up our own Twitter application. We used the `tweepy` library to create an API object for gathering tweet data:
```python
import tweepy

consumer_key = 'hidden'
consumer_secret = 'hidden'
access_token = 'hidden'
access_secret = 'hidden'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
```

After quering each tweet ID, we wrote its JSON data to the `tweet_json.txt` file with each tweet's JSON data on its own line. We then read this file, line by line, to create the `dft` DataFrame with required data extracted.

## Assess Data
Key assumptions of assessing data are as follows:
* We only want original ratings (no retweets or replies) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time. Therefore, we will assess and clean 8 quality issues and 3 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* We will gather the additional tweet data only for tweets in the *twitter_archive_enhanced.csv* dataset.

With these assumptions in mind we assessed the input data and identified these issues:

**Quality issues**
- Q1. Replies are not original tweets.
- Q2. Retweets are not original tweets.
- Q3. Some tweets don't have any image
- Q4. Some ratings are incorrectly identified
- Q5. Some ratings are missing
- Q6. Names starting with lowercase are incorrect
- Q7. Names with value None are incorrect
- Q8. Column timestamp has the dtype object (string)

**Tidiness issues**
- T1. Dogs' stages (doggo, pupper, puppo, floofer) as columns
- T2. Multiple image predictions in one row
- T3. Data in multiple datasets

Concerning quality, Q1, Q2, Q3 and Q5 are **completeness** issues in a sense that there is data out of the scope of this project. Q8 is a **validity** issue. Q4, Q6, and Q7 are **accuracy** issues.

In [1]:
%%html
<style>
table {float:left}
</style>

## Clean Data

Before cleaning we made copies of input data:
```python
dfa_clean = dfa.copy() # archive
dfi_clean = dfi.copy() # image predictions
dft_clean = dft.copy() # data from Twitter API
```

Here is a toolbox for cleaning efforts:

Issue | Code | Test
--- | --- | ---
Q1 | Filter rows out by df.isna(); Drop cols by df.drop()  | df.info()
Q2 | ditto | ditto
Q3 | Filter rows out by df.apply() | df.apply()
Q4 | Update values by df.at[] | df.loc[]
Q5 | Drop rows by df.drop() | df.index.intersection()
Q6 | Replace values by df.apply() | re.findall()
Q7 | ditto | df.query()
Q8 | Convert dtype by pd.to_datetime() | df.dtype
T1 | Derive a new col by df.apply(func, axis=1) | df.value_counts(), df.info()
T2 | ditto | df.info()
T3 | Merge datasets by df.merge() | df.info(), df.head()

The next diagram displays transformations used to provide a tidy clean dataset:

![Data Cleaning](http://www.plantuml.com/plantuml/png/pLHVQzim47_Nfn1z62saXIQb3oEOmie-mRQNCOnYdJqL_J7IgKqO-Uuxso5IOeCtmaRVV7S_t_r-fxjx2YWyHgEPqDgz6RLLjc5WKU6EorhmzVfPeFZiXO6YuByOlW5WgIJ_o9N5nmUcREcXrRiINKd9C8OXMcidpkCnW3-DeZ90VvcMV6xp0XMouAAl8Da8MqoMXvu1oAbYu-YeuXWoGG6shR2I8D7haDKMIDqfRJH0dyuVUt08IB3EA7iIii9aMgHh6fTCukHgyCdHnhOzjpFyFhsCcd-PbxRf1AJVbUd97m7jVLRflgoShRD6aOVeacFZd6RjCiCkHzZb2NQLOLSZx2h7Pf9mHb9wF4ePBHwzjNXrNY6SkVUCLLg4mAt3aW2DiaANVO90fVZryyA3lE5TgGl7-BzCvPvr2tbxMqm9whIwAFdL-WDw4NuMLrm4td-oAVrtTyKXUcWIvIvVXG-Tm4g3iDIYws-B7NHtuQRlqPZJvIkHL8qPJ553BsoedJS3FKTc0mEzIvThJ2IlbyYeFo7pFdQBo7QAfacpj7f3l-VtBvy0AH0w-_7XkmqB_lNBfn_Jj_lVdrMYr2Hp_l1jF81CuR-yAkDD6Ley66nwMdC8vUeP4rJDGtko7VIy39uwdi3AQFHl)

## Store Data
Even if a Jupyter notebook ensures repeatability of the wrangling process, it is recommended to save the cleaned dataset for further processing. It is beneficial especially in light of the amount of time needed for quering the Twitter API.

We saved the cleaned dataset to both a CSV file and a SQlite database using this code:
```python
# Store the dataframe for further processing in CSV format
with open('twitter_archive_master.csv', 'w') as file:
  df_clean.to_csv(file)

# Store the dataframe for further processing in a database
from sqlalchemy import create_engine

# Create SQLAlchemy engine and empty database
engine = create_engine('sqlite:///weratedogsdata_clean.db')

# Store dataframes in database
df_clean.to_sql('df_clean', engine, index=False)
```

To restore data use this code (provided that the archive had been uploaded to a GitHub repository):

```python
# Restore dataframe for further processing
import requests
from sqlalchemy import create_engine

# Upload the database from GitHub
url_db = 'https://github.com/lustraka/Data_Analysis_Workouts/blob/main/Analyse_Twitter_Data/weratedogsdata_clean.db?raw=true'
r = requests.get(url_db)
with open('weratedogsdata_clean.db', 'wb') as file:
  file.write(r.content)

# Create SQLAlchemy engine and connect to the database
engine = create_engine('sqlite:///weratedogsdata_clean.db')

# Read dataframes from SQlite database
df_clean = pd.read_sql('SELECT * FROM df_clean', engine)
df_clean.shape
```


## Result: WeRateDogs Master Data Set
The description of the resulting dataset is as follows:

| # | Variable | Non-Null | Nunique | Dtype | Notes |
|---|----------|----------|---------|-------|-------|
| 0 | tweet_id | 1657 | 1657 | int64 | The Tweet's unique identifier .|
| 1 | timestamp | 1657 | 1657 | datetime64[ns, UTC] | Time when this Tweet was created. |
| 2 | source | 1657 | 3 | object | Utility used to post the Tweet. |
| 3 | text | 1657 | 1657 | object | The actual text of the status update. |
| 4 | expanded_urls | 1657 | 1657 | object | The URLs of the Tweet's photos. |
| 5 | rating_numerator | 1657 | 26 | int64 | The rating numerator extracted from the text. |
| 6 | rating_denominator | 1657 | 10 | int64 | The rating denominator extracted from the text. |
| 7 | name | 1657 | 831 | object | The dog's name extracted from the text. |
| 8 | stage | 1657 | 5 | object | The dog's stage extracted from the text.|
| 9 | jpg_url | 1657 | 1657 | object | The URL of the image used to classify the breed of dog. |
| 10 | img_num | 1657 | 4 | int64 | The image number that corresponded to the most confident prediction.  |
| 11 | breed | 1657 | 113 | object | The most confident classification of the breed of dog predicted from the image. |
| 12 | retweet_count | 1657 | 1352 | int64 | Number of times this Tweet has been retweeted. |
| 13 | favorite_count | 1657 | 1561 | int64 | Indicates approximately how many times this Tweet has been liked by Twitter users. |

## Caveats
- The cleaned dataset has 1657 observations starting at the November 15th, 2015 when the WeRateDogs Twitter account was launched and ending at the August 17th, 2017 when the archive was exported.
- Variables *rating_numerator, rating_denominator, name,* and *stage* was extracted from the tweet's text. The rating is a part of a humorous aspect of the content. There is hardly any value in analysing these variables.
- The variable *breed* is inferred from the image using machine learning algorithm. We can use this variable keeping on mind that there can be some inaccuracies.
- The variables *favorite_count*, and *retweet_count* reflects the preferences of Twitter users. We can use these variables keeping in mind they come from a non-random sample of human population.